In [1]:
import pdb
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
import pylab as plt
from astropy.wcs import WCS
from astropy.io import fits
import readcol
from utils import shift_twod
from utils import dist_idl
from utils import loggen
from utils import gauss_kern
from lmfit import Parameters, minimize, fit_report
from simstack.simstack import stack_libraries_in_redshift_slices
from skymaps import Skymaps
from skymaps import Field_catalogs

%matplotlib inline  

In [2]:
popcolor=['blue','red','green','orange','black','grey','chocolate','darkviolet','pink',
          'magenta','dodgerblue','lavender','blue','red','green','orange','black','grey',
          'chocolate','darkviolet','pink','magenta','dodgerblue','lavender']

In [3]:
#Define Redshift Bins
z_lo = np.array([0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5])
z_hi = np.array([0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0])
z_nodes = np.array([0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0])
z_mid = (z_nodes[:-1] + z_nodes[1:])/2
nz = len(z_nodes) - 1

In [4]:
#Define Stellar Mass Bins
m_lo = np.array([8.5, 9.5,10.0,10.5,11.0])
m_hi = np.array([9.5,10.0,10.5,11.0,13.0])
m_nodes = np.array([8.5, 9.5,10.0,10.5,11.0,13.0])
m_mid = (m_nodes[:-1] + m_nodes[1:])/2
nm = len(m_nodes) - 1

In [5]:
#Decide which Maps to include
wv0 = np.array([1,1,1,1,1,1]) # This includes all maps
#wv0 = np.array([1,0,1,0,1,1]) # This one only includes the first, third, fifth, and sixth
indstack = np.where(wv0 == 1)

# Need to Edit next panel for your own use 
### Look for Maps below in www.astro.caltech.edu/viero/Jason/cosmos/

In [6]:
## Map Directories 

dir_spitzer_maps = '/Users/marco/Code/Python/Modules/simstack/data/cutouts/'
dir_spire_maps = '/Users/marco/Code/Python/Modules/simstack/data/cutouts/'
dir_pacs_maps = '/Users/marco/Code/Python/Modules/simstack/data/cutouts/'
dir_aztec_maps = '/Users/marco/Code/Python/Modules/simstack/data/'

## Dictionary Names
library_keys =['mips24'
               ,'pacs_green'
               ,'pacs_red'
               ,'spire_PSW'
               ,'spire_PMW'
               ,'spire_PLW'
               ,'aztec'
              ]

wavelength=[24.,100,160,250,250,350,500,1100]
nwv = np.sum(wv0) 
fwhm =[6.32,7.4, 11.3,18.1, 25.2, 36.6,18.]
efwhm=[6.32,6.7, 11.2,17.6, 23.9, 35.2,18.] # want to the measured effective FWHM later
color_correction=[1.25,23.58,1.018,1.018,0.9914,0.95615,1.0]
beam_area = [1.54756e-09,1.,1.,1.,1.,1.,1.] #sr

pixsize_suffix = '4.0_arcsec_pixels'

maps = [dir_spitzer_maps+'mips_24_GO3_sci_10.cutout.fits' 
        ,dir_pacs_maps+'pep_COSMOS_green_Map.DR1.sci.cutout.fits'
        ,dir_pacs_maps+'pep_COSMOS_red_Map.DR1.sci.cutout.fits'
        ,dir_spire_maps+'cosmos-uvista-hipe12_itermap_10_iterations_'+pixsize_suffix+'_PSW.signal.cutout.fits'
        ,dir_spire_maps+'cosmos-uvista-hipe12_itermap_10_iterations_'+pixsize_suffix+'_PMW.signal.cutout.fits'
        ,dir_spire_maps+'cosmos-uvista-hipe12_itermap_10_iterations_'+pixsize_suffix+'_PLW.signal.cutout.fits'
        ,dir_aztec_maps+'cosmos_jcmt_kscott20100925_map.cutout.fits'
       ]
noises = [dir_spitzer_maps+'mips_24_GO3_unc_10.cutout.fits' 
        ,dir_pacs_maps+'pep_COSMOS_green_Map.DR1.err.cutout.fits'
        ,dir_pacs_maps+'pep_COSMOS_red_Map.DR1.err.cutout.fits'
        ,dir_spire_maps+'cosmos-uvista-hipe12_itermap_10_iterations_'+pixsize_suffix+'_PSW.noise.cutout.fits'
        ,dir_spire_maps+'cosmos-uvista-hipe12_itermap_10_iterations_'+pixsize_suffix+'_PMW.noise.cutout.fits'
        ,dir_spire_maps+'cosmos-uvista-hipe12_itermap_10_iterations_'+pixsize_suffix+'_PLW.noise.cutout.fits'
        ,dir_aztec_maps+'cosmos_jcmt_kscott20100925_noise.cutout.fits'
       ]

# Reading Maps into Objects, and Objects into Dictionaries

In [7]:
#Different libraries for Actual and Gaussian Approximated PSFs
sky_library_gaussians={}

for t in indstack[0]:
    sky_library_gaussians[library_keys[t]] = Skymaps(maps[t],noises[t],efwhm[t],color_correction=color_correction[t])
    sky_library_gaussians[library_keys[t]].add_wavelength(wavelength[t])
    sky_library_gaussians[library_keys[t]].add_fwhm(efwhm[t]) 
    if beam_area[t] != 1: sky_library_gaussians[library_keys[t]].beam_area_correction(beam_area[t])

/Users/marco/Code/Python/Modules/Utils/utils.py:189: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  delt = zeros([side,side])


## Catalog Directories and Names
### Look for Maps below in www.astro.caltech.edu/viero/Jason/cosmos/

In [8]:
path_catalog = '/data/maps_cats_models/catalogs/UVISTA/DR2/'
#Catalog must have the following (case sensitive) Columns:
# - RA
# - DEC
# - z_peak
# - LMASS
# - rf_U_V
# - rf_V_J
file_catalog = 'UVISTA_DR2_master_v2.1_USE.csv'

In [9]:
tbl = pd.read_table(path_catalog+file_catalog,sep=',')
tbl['LMASS']=tbl['lmass']

In [10]:
uVista = Field_catalogs(tbl)

In [11]:
# This function creates a column 'sfg' based on its UVJ colors, where sf == 1 and qt == 0
uVista.separate_sf_qt()

In [ ]:
#Stack in redshift bins, with Layers divided by Stellar Mass and Star-Forming/Quiescent Galaxies
pop = ['sf','qt']
npop=2
all_stacked_fluxes_psfs = np.zeros([nwv,nz,nm,npop])
all_luminosity_temp_psfs= np.zeros([nz,nm,npop,2])

for iz in range(nz):
    zn = z_nodes[iz:iz+2]
    uVista.get_sf_qt_mass_redshift_bins(zn,m_nodes)
    radec_m_z_p = uVista.subset_positions(uVista.id_z_ms)
    stacked_fluxes_psfs =  None
    n_sources_max = None
    
    stacked_fluxes_psfs = stack_libraries_in_redshift_slices(
        sky_library_gaussians,
        radec_m_z_p)
    
    #pdb.set_trace()
    args = radec_m_z_p.keys()
    for iwv in range(nwv):
        stacked_fluxes_wv = stacked_fluxes_psfs[str(wavelength[indstack[0][iwv]])]
        for j in range(nm):
            for k in range(npop):
                arg = 'z_'+str(zn[0])+'-'+str(zn[1])+'__m_'+str(m_nodes[j])+'-'+str(m_nodes[j+1])+'_'+pop[k]
                all_stacked_fluxes_psfs[iwv,iz,j,k] = stacked_fluxes_wv[arg.replace('.','p').replace('-','_')].value
    
    #pdb.set_trace()
    #PLOT
    plt.figure()
    plt.ylim([-1e-1,3e0])
    plt.ylim([1e-2,9e1])
    plt.xlim([90,510])
    plt.xlim([90,1150])
    plt.yscale('log')
    plt.ylabel('Flux Density [mJy]',fontsize=14)
    plt.xlabel('Wavelength [microns]',fontsize=14)
    for j in range(nm): 
        for k in range(npop):
            plt.plot(np.array(wavelength)[indstack],1e3*all_stacked_fluxes_psfs[:,iz,j,k],color=popcolor[npop*j+k],label=[pop[k],str(m_mid[j])])
            plt.legend()
    plt.show()
    #pdb.set_trace()

/Users/marco/Code/Python/Modules/Utils/utils.py:79: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  outmap = np.zeros([pad_side, pad_side])
/Users/marco/Code/Python/Modules/Utils/utils.py:161: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = np.zeros([float(n1),float(m1)])
/Users/marco/Code/Python/Modules/Utils/utils.py:167: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a[:,i] = y
/Users/marco/Code/Python/Modules/Utils/utils.py:169: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a[:,m1-i]=y
/Users/marco/Code/Python/Modules/Utils/utils.py:83: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  circ = np.zeros([pad_side, pad_side])


stacking spire_PSW
stacking pacs_green

In [ ]:
for iwv in range(len(wavelength)):
    plt.figure()
    for j in range(nm):
        plt.yscale('log')
        plt.title(str(wavelength[iwv]))
        plt.plot(z_mid,1e3*(all_stacked_fluxes_psfs[iwv,:,j,0]),'o')
        plt.plot(z_mid,1e3*(all_stacked_fluxes_psfs[iwv,:,j,0]))
    plt.show()